In [1]:
import numpy as np
import pandas as pd

### initial data

In [2]:
t = pd.read_csv('../csv/itree_qexactive_data.csv').drop('25 HCD', axis=1)

### 65 HCD have na, so just dropna()

In [3]:
df = t.dropna()
df['Precursor m/z'] = round(df['Precursor m/z'], 2)

/Users/yoshitaka-i/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [4]:
### after dropping shape
df.shape

(18577, 14)

### divide pos and neg

In [5]:
df[df.Mode == 'positive'].reset_index(drop=True).to_csv('../csv/pos.csv')
df[df.Mode != 'positive'].reset_index(drop=True).to_csv('../csv/neg.csv')

In [6]:
pos = pd.read_csv('../csv/pos.csv', index_col=0)
neg = pd.read_csv('../csv/neg.csv', index_col=0)

### drop_duplicates by InChIKey to get InChIKey list

In [7]:
pos.drop_duplicates('InChIKey').InChIKey.to_csv('../csv/pos_key.csv')
neg.drop_duplicates('InChIKey').InChIKey.to_csv('../csv/neg_key.csv')

### make class's list ( just merge pos's class and neg's class

In [8]:
pd.concat([
    pd.read_csv('../csv/pos_class.csv', index_col=None), 
    pd.read_csv('../csv/neg_class.csv', index_col=None)]
).drop_duplicates('InChIKey').reset_index(drop=True).to_csv('../csv/class.csv')

### merge class and ms data by InChIKey

In [9]:
pos = pd.merge(pos, pd.read_csv('../csv/class.csv', index_col=0), on='InChIKey')
neg = pd.merge(neg, pd.read_csv('../csv/class.csv', index_col=0), on='InChIKey')

In [10]:
print(pos.shape)
print(neg.shape)

(14223, 24)
(4354, 24)


### drop_duplicates by precursor and InChIKey

In [11]:
pos = pos.drop_duplicates(subset=['Precursor m/z', 'InChIKey'])
neg = neg.drop_duplicates(subset=['Precursor m/z', 'InChIKey'])

In [12]:
print(pos.shape)
print(neg.shape)

(6791, 24)
(2051, 24)


### just choose 5 colomns to use as feature

In [13]:
pos[['35 HCD','45 HCD','65 HCD','Class','Subclass']].reset_index(drop=True).to_csv('../csv/pos_for_ml.csv')
neg[['35 HCD','45 HCD','65 HCD','Class','Subclass']].reset_index(drop=True).to_csv('../csv/neg_for_ml.csv')